In [1]:
!pip install numba

In [6]:
!pip install pympler==0.4.3


  Using cached Pympler-0.4.3-py3-none-any.whl
  Attempting uninstall: pympler
    Found existing installation: Pympler 1.0.1
    Uninstalling Pympler-1.0.1:
      Successfully uninstalled Pympler-1.0.1


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper 
from pycox.evaluation import EvalSurv

from pycox.models import CoxTime
import torch # For building the networks 
from sklearn.model_selection import KFold, GridSearchCV
from estimators import CoxTimeEstimator
import torchtuples as tt 
from collections import defaultdict

LookupError: unknown encoding: unicode-internal